In [41]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [42]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from openpyxl_image_loader import SheetImageLoader

In [43]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [44]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [45]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [46]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [47]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['select_options']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [48]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [49]:
def get_value_multiple(field, occ, value):
    
    results=[]
    
    if value[-1] == '.':
        values=value.split('.')[:-1]
    else:
        values=value.split('.')
    
    for item in values:
        results.append(get_value(field, occ, item.capitalize()))
    
    return results

In [50]:
!cat .env

STAGE=PRODUCTION


In [51]:
filename = 'Entrevias_Drenagem_Profunda ( kartado Existente )'

In [67]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]

In [68]:
image_loader = SheetImageLoader(ws)

for index, row in enumerate(ws.rows):
    if index == 0:
        continue
    
    if row[0].value is not None:
        
        try:
            image = image_loader.get('D' + str(index+1))
            image2 = image_loader.get('H' + str(index+1))
        except:
            print(row[0])

<Cell 'Profunda (Ex)'.A5>
<Cell 'Profunda (Ex)'.A7>
<Cell 'Profunda (Ex)'.A18>
<Cell 'Profunda (Ex)'.A19>
<Cell 'Profunda (Ex)'.A108>
<Cell 'Profunda (Ex)'.A120>
<Cell 'Profunda (Ex)'.A123>
<Cell 'Profunda (Ex)'.A129>
<Cell 'Profunda (Ex)'.A147>
<Cell 'Profunda (Ex)'.A150>
<Cell 'Profunda (Ex)'.A157>
<Cell 'Profunda (Ex)'.A163>
<Cell 'Profunda (Ex)'.A168>
<Cell 'Profunda (Ex)'.A194>
<Cell 'Profunda (Ex)'.A205>
<Cell 'Profunda (Ex)'.A206>
<Cell 'Profunda (Ex)'.A211>
<Cell 'Profunda (Ex)'.A212>
<Cell 'Profunda (Ex)'.A216>
<Cell 'Profunda (Ex)'.A218>
<Cell 'Profunda (Ex)'.A219>
<Cell 'Profunda (Ex)'.A221>
<Cell 'Profunda (Ex)'.A230>
<Cell 'Profunda (Ex)'.A233>
<Cell 'Profunda (Ex)'.A236>
<Cell 'Profunda (Ex)'.A238>
<Cell 'Profunda (Ex)'.A249>
<Cell 'Profunda (Ex)'.A263>
<Cell 'Profunda (Ex)'.A295>
<Cell 'Profunda (Ex)'.A296>
<Cell 'Profunda (Ex)'.A321>
<Cell 'Profunda (Ex)'.A330>
<Cell 'Profunda (Ex)'.A346>
<Cell 'Profunda (Ex)'.A415>
<Cell 'Profunda (Ex)'.A416>
<Cell 'Profunda (Ex)'.A422

In [1]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
        
        for i,a in enumerate(obj['img']):
            try:
                a.name='Foto '+i
            except:
                pass

        
        values.append(obj)

NameError: name 'ws' is not defined

In [3]:
company=Company.objects.get(name='Entrevias')
firm=Firm.objects.get(company=company, name='Programação e Controle')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')

occurrence_type_inventory=OccurrenceType.objects.get(uuid='8f892e14-bd36-4712-81ba-1f34caae423e')
occurrence_type_reporting=OccurrenceType.objects.get(uuid='ca224c1d-3ce0-4948-8029-1401feb999ec')
user=User.objects.get(username='rlcs')
company,firm,occurrence_type_inventory,occurrence_type_reporting

(<Company: 91d69c92-2284-4570-9011-760a5a74a708: Entrevias>,
 <Firm: [91d69c92-2284-4570-9011-760a5a74a708: Entrevias] 0dc0862e-579b-40af-aa51-8cff6de82d89: Programação e Controle>,
 <OccurrenceType: Drenagem Profunda (novo) - ['Entrevias']>,
 <OccurrenceType: Monitoramento de Drenagem Profunda (novo) - ['Entrevias']>)

In [71]:
len(values)

623

In [72]:
values[2]

{'ID do Elemento': 'BU 333 SP 449+290 L 1',
 'Rodovia': 'SP 333',
 'km': '449+290',
 'Foto_1': 'Drenagem_BU_449+290_00927_EntreVias_20220414_154154.jpg',
 'Data Foto_1': None,
 'Tipo Foto_1': None,
 'Descrição Foto_1': None,
 'Foto_2': 'Drenagem_BU_449+335_00927_EntreVias_20220414_152828.jpg',
 'Data Foto_2': None,
 'Tipo Foto_2': None,
 'Descrição Foto_2': None,
 'Tipo montante': 'ALA',
 'Diâmetro montante': 'ø1',
 'Largura montante': None,
 'Altura montante': None,
 'Comprimento montante': 39.8,
 'Cota 1 montante': None,
 'Cota 2 montante': None,
 'Latitude montante': -22.89731128,
 'Longitude montante': -50.77110619,
 'Sentido montante': 'LESTE',
 'Faixa montante': None,
 'Diagnóstico montante': '-',
 'Tipo': 'ALA',
 'Diâmetro': 'ø1',
 'Largura': None,
 'Altura': None,
 'Comprimento': 39.8,
 'Cota 1': None,
 'Cota 2': None,
 'Latitude': -22.89719851,
 'Longitude': -50.77147492,
 'Sentido': 'OESTE',
 'Faixa': None,
 'Diagnóstico': '-',
 'Conceito': None,
 'img': [<openpyxl.drawing.im

In [73]:
objects_reporting=[]

for index, a in enumerate(tqdm(values)):
    try:
        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido montante'].capitalize()))
    except Exception as err:
        direction='5'
    
    point, road = get_road_coordinates(a['Rodovia'].strip().replace(' ', '-'), round(float(a['km'].replace('+', '.')), 3), direction, company)

    try:
        objects_reporting.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type_reporting,
                    lane=get_connected_reference(company, 'reporting', 'lane', 'Faixa de domínio'),
                    road=road,
                    road_name=road.name,
                    direction=direction,
                    created_by=user,
                    km=round(float(a['km'].replace('+', '.')), 3),
                    parent=parent,
                    point=point,
                    status=status,
                    approval_step=step,
                    firm=firm,
                    form_data={
                        'elementid': a['ID do Elemento'],
                        'upstream_type': get_value('Tipo Montante', occurrence_type_reporting, a['Tipo montante']) if a['Tipo montante'] is not None and a['Tipo montante'] != '-' else None,
                        'upstream_diameter': get_value('Diâmetro Montante', occurrence_type_reporting, a['Diâmetro montante'].split('ø')[1]) if a['Diâmetro montante'] is not None and a['Diâmetro montante'] != '-' else None,
                        'upstream_length': round(float(str(a['Comprimento montante']).replace(',', '.')), 3) if a['Comprimento montante'] is not None and a['Comprimento montante'] != '-' else None,
                        'upstream_width': round(float(str(a['Largura montante']).replace(',', '.')), 3) if a['Largura montante'] is not None and a['Largura montante'] != '-' else None,
                        'upstream_height': round(float(str(a['Altura montante']).replace(',', '.')), 3) if a['Altura montante'] is not None and a['Altura montante'] != '-' else None,
                        'upstream_lat': float(a['Latitude montante']) if a['Latitude montante'] is not None and a['Latitude montante'] != '-' else None,
                        'upstream_long': float(a['Longitude montante']) if a['Longitude montante'] is not None and a['Longitude montante'] != '-' else None,
                        'downstream_type': get_value('Tipo', occurrence_type_reporting, a['Tipo']) if a['Tipo'] is not None and a['Tipo'] != '-' else None,
                        'downstream_diameter': get_value('Diâmetro', occurrence_type_reporting, a['Diâmetro'].split('ø')[1]) if a['Diâmetro'] is not None and a['Diâmetro'] != '-' else None,
                        'downstream_length': round(float(str(a['Comprimento']).replace(',', '.')), 3) if a['Comprimento'] is not None and a['Comprimento'] != '-' else None,
                        'downstream_width': round(float(str(a['Largura']).replace(',', '.')), 3) if a['Largura'] is not None and a['Largura'] != '-' else None,
                        'downstream_height': round(float(str(a['Altura']).replace(',', '.')), 3) if a['Altura'] is not None and a['Altura'] != '-' else None,
                        'downstream_lat': float(a['Latitude']) if a['Latitude'] is not None and a['Latitude'] != '-' else None,
                        'downstream_long': float(a['Longitude']) if a['Longitude'] is not None and a['Longitude'] != '-' else None,
                        'downstream_direction': get_value('Sentido', occurrence_type_reporting, a['Sentido'].capitalize()) if a['Sentido'] is not None and a['Sentido'] != '-' else None,
                        'downstream_lane': a['Faixa'] if a['Faixa'] is not None and a['Faixa'] != '-' else None,
                        'upstream_first_quota': float(a['Cota 1 montante']) if a['Cota 1 montante'] is not None and a['Cota 1 montante'] != '-' else None,
                        'upstream_second_quota': float(a['Cota 2 montante']) if a['Cota 2 montante'] is not None and a['Cota 2 montante'] != '-' else None,
                        'upstream_diag': get_value_multiple('Diagnóstico Montante', occurrence_type_reporting, a['Diagnóstico montante']) if a['Diagnóstico montante'] is not None and a['Diagnóstico montante'] and a['Diagnóstico montante'] != '-' else None,
                        'downstream_first_quota': float(a['Cota 1']) if a['Cota 1'] is not None and a['Cota 1'] != '-' else None,
                        'downstream_second_quota': float(a['Cota 2']) if a['Cota 2'] is not None and a['Cota 2'] != '-' else None,
                        'downstream_diag': get_value_multiple('Diagnóstico', occurrence_type_reporting, a['Diagnóstico']) if a['Diagnóstico'] is not None and a['Diagnóstico'] and a['Diagnóstico'] != '-' else None                        
                        }
                    ), a['img']))
        
    except Exception as e:
#         exception_type, exception_object, exception_traceback = sys.exc_info()
#         line_number = exception_traceback.tb_lineno
        print(f'AQUI: {e} - {index+2}')
        pass

  0%|          | 0/623 [00:00<?, ?it/s]

In [74]:
for reporting, images in tqdm(objects_inventory):
    reporting.save()
#     print(reporting.number)

  0%|          | 0/623 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
'NoneType' object has 

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has 

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has 

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM witho

...

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM 

Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direct

Searching KM on direction...
Searching KM without direction...
Found KM on Road 317: SPA-274/333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 317: SPA-274/333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM wit

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 280: SP-266
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 300: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
'NoneType' object has 

Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direc

In [75]:
for reporting, images in tqdm(objects_inventory):
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/623 [00:00<?, ?it/s]

In [76]:
for reporting, images in tqdm(objects_reporting):
    reporting.save()
#     print(reporting.number)

  0%|          | 0/623 [00:00<?, ?it/s]

Searching KM on direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 271: SP-333
Searching KM on direction...
Found KM on Road 280: SP-266
Searching KM on direction...
Searching KM without direction...
Found KM on Road 280: SP-266
Searching KM on direction...
Searching KM without direction...
Found KM on Road 280: SP-266
Searching KM on direction...
Searching KM without direction...
Found KM on Road 280: SP-266
Searching KM on direction...
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Found KM on R

Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direct

Searching KM on direction...
Searching KM without direction...
Found KM on Road 318: SPA-409/333
Searching KM on direction...
Found KM on Road 318: SPA-409/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 318: SPA-409/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 318: SPA-409/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Found KM on Road 287: SPA-245/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 277: SPA-245/333
Searching KM on direction...
Found KM on Road 277: SPA-245/333
Search

Searching KM on direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 299: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 296: SP-294
Searching KM on direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333
Searching KM on direction...
Found KM on Road 277: SPA-245/333
Searching KM on direction...
Found KM on Road 277: SPA-245/333
Searching KM on direction...
Searching KM without direction...
Found KM on Road 282: SP-333


In [77]:
for reporting, images in tqdm(objects_reporting):
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/623 [00:00<?, ?it/s]

In [89]:
reps=[a[0] for a in objects]

In [90]:
photos=ReportingFile.objects.filter(reporting__in=reps)
photos.count()

20

In [91]:
# photos.delete()

(20, {'reportings.ReportingFile': 20})